# Chapter 7: Building and AutoEncoder

Generative models are the most promising push toward enabling computers to have an understanding of the world. They are true unsupervised models, and are able to perform those tasks that many today consider to be at the cutting edge of Artificial Intelligence (AI). Generative models are different for precisely the reason as it sounds: they generate data. Centered mostly around computer vision tasks, this class of network has the power to create new faces, new handwriting, or even paintings. 

In [1]:
import numpy as np
import tensorflow as tf

/Users/patricksmith/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Building an AutoEncoder

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

#### Define the weight and bias initializer. 

In [ ]:
initializer = tf.contrib.layers.xavier_initializer()

#### Define the Encoder

In [ ]:
def encoder(x):
    input_layer = tf.layers.dense(inputs=x, units=784, activation=tf.nn.relu,
                                 kernel_initializer=initializer, bias_initializer=initializer 
                                 )
    z_prime = tf.layers.dense(inputs=input_layer, units=256, activation=tf.nn.relu,
                             kernel_initializer=initializer, bias_initializer=initializer
                             )
    z = tf.layers.dense(inputs=z_prime, units=128, activation=tf.nn.relu,
                       kernel_initializer=initializer, bias_initializer=initializer
                       )
    return z

#### Define the Decoder

In [ ]:
def decoder(x):
    x_prime_one = tf.layers.dense(inputs=x, units=128, activation=tf.nn.relu,
                                 kernel_initializer=initializer, bias_initializer=initializer
                                 )
    x_prime_two = tf.layers.dense(inputs=x_prime_one, units=256, activation=tf.nn.relu,
                                 kernel_initializer=initializer, bias_initializer=initializer
                                 )
    output_layer = tf.layers.dense(inputs=x_prime_two, units=784, activation=tf.nn.relu,
                                  kernel_initializer=initializer, bias_initializer=initializer
                                  )
    return output_layer

#### Training Parameters

In [ ]:
display = 1
input_dim = 784 
learning_rate = 0.001
num_steps = 30000
batch_size = 256

#### Input placeholder

In [ ]:
x = tf.placeholder("float", [None, input_dim])

#### Construct the encoder, decoder, and optimizer

In [ ]:
# Construct the full autoencoder
z = encoder(x)

## x_prime represents our predicted distribution
x_prime = decoder(z) 

# Define the loss function and the optimizer
loss = tf.reduce_mean(tf.pow(x - x_prime, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

#### Run the training cycle

In [ ]:
## Begin a new tensorflow session:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    ## Training Loop
    for i in range(1, num_steps+1):
    
        ## Feed Batches of MNIST Data
        batch_x, _ = mnist.train.next_batch(batch_size)

        ## Run the Optimization Process
        _, l = sess.run([optimizer, loss], feed_dict={x: batch_x})

        ## Display the loss at every 1000 out of 30,000 steps
        if i % display == 0 or i == 1:
            print('Step %i: Loss: %f' % (i, l))

    n = 4
    canvas_orig = np.empty((28 * n, 28 * n))
    canvas_recon = np.empty((28 * n, 28 * n))

    for i in range(n):

        batch_x, _ = mnist.test.next_batch(n)

        # Encode and decode each individual written digit
        g = sess.run(decoder, feed_dict={x: batch_x})

        # Display original images
        for j in range(n):

            # Draw the original digits
            canvas_orig[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = batch_x[j].reshape([28, 28])

        # Display reconstructed images
        for j in range(n):

            # Draw the reconstructed digits
            canvas_recon[i * 28:(i + 1) * 28, j * 28:(j + 1) *                                        28] = g[j].reshape([28, 28])

    # Plot the original image vs the reconstructed images. 
    print("Original Images")
    plt.figure(figsize=(n, n))
    plt.imshow(canvas_orig, origin="upper", cmap="gray")
    plt.show()

    print("Reconstructed Images")
    plt.figure(figsize=(n, n))
    plt.imshow(canvas_recon, origin="upper", cmap="gray")
    plt.show()